In [7]:
!pip install  -q -r req.txt

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [13]:
from dotenv import load_dotenv
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

load_dotenv('../../.env')

from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
import re
from langchain_community.document_loaders import PyPDFLoader
from pdfminer.high_level import extract_text

from transformers import AutoTokenizer, DistilBertForQuestionAnswering, DistilBertTokenizerFast

tokenizer = AutoTokenizer.from_pretrained('deepset/roberta-base-squad2')
model = AutoModelForQuestionAnswering.from_pretrained('deepset/roberta-base-squad2')

text = extract_text("./Computer_vision.pdf")
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
docs = text_splitter.create_documents([text])
db3 = Chroma(persist_directory="./chroma_db").from_documents(docs)


In [15]:
db3 = Chroma(persist_directory="./chroma_db").from_documents(docs)


Using embedded DuckDB with persistence: data will be stored in: ./chroma_db
No embedding_function provided, using default embedding function: SentenceTransformerEmbeddingFunction
Using embedded DuckDB without persistence: data will be transient
No embedding_function provided, using default embedding function: SentenceTransformerEmbeddingFunction


In [16]:
retriever = db3.as_retriever()
pages = retriever.get_relevant_documents("What is computer vision ?")
context = ""
import re 
for p in pages: 
    context +=  '\n' +  re.sub('[\(\[].*?[\)\]]', "", p.page_content)
    context = context.replace('"', '')
print (context)


computer  vision  seeks  to  apply  its  theories  and  models  for  the  construction  of  computer  vision  systems.
Machine vision refers to a systems engineering discipline, especially in the context of factory automation, In
more recent times the terms computer vision and machine vision have converged to a greater degree.: 13
Computer vision is an interdisciplinary field that deals with how computers can be made to gain high-level
understanding from digital images or videos. From the perspective of engineering, it seeks to automate tasks
that the human visual system can do. Computer vision is concerned with the automatic extraction,
analysis and understanding of useful information from a single image or a sequence of images. It involves
the development of a theoretical and algorithmic basis to achieve automatic visual understanding. As a
scientific  discipline,  computer  vision  is  concerned  with  the  theory  behind  artificial  systems  that  extract
information  from  image

<>:6: SyntaxWarning: invalid escape sequence '\('
<>:6: SyntaxWarning: invalid escape sequence '\('
/var/folders/n5/sr7wwb8n1wggd7jsk_k37l_w0000gn/T/ipykernel_30754/3155017363.py:6: SyntaxWarning: invalid escape sequence '\('
  context +=  '\n' +  re.sub('[\(\[].*?[\)\]]', "", p.page_content)


In [1]:
question = "what is the urban agglomeration of Bangalore?"
context = '''
Bangalore is a megacity with a population of 8,443,675 in the city and 10,456,000 in the urban agglomeration,[6][5] up from 8.5 million at the 2011 census.[115] It is the third most populous city in India, the 18th most populous city in the world and the fifth most populous urban agglomeration in India.[11][116] With a growth rate of 38% during the decade, Bangalore was the fastest-growing Indian metropolis after New Delhi between 1991 and 2001. Residents of Bangalore are referred to as "Bangaloreans" in English, Bengaloorinavaru or Bengaloorigaru in Kannada and Banglori in Hindi or Urdu.[117] People from other states have migrated to Bangalore, study, or work there as well.[118]
'''
def query(context, question, verbose=True, context_max_len=2500):
    if len(context) > context_max_len:
        if verbose:
            print(f'Warning: Context length {len(context)} exceeds max {context_max_len}, truncating')
            context = context[:context_max_len]
    encoding = tokenizer.encode_plus(question, context)
    input_ids, attention_mask = encoding["input_ids"], encoding["attention_mask"]
    ret = model(torch.tensor([input_ids]), attention_mask=torch.tensor([attention_mask]))  # bug work-around
    start_scores, end_scores = ret[0], ret[1]
    ans_tokens = input_ids[torch.argmax(start_scores) : torch.argmax(end_scores)+1]
    answer_tokens = tokenizer.convert_ids_to_tokens(ans_tokens , skip_special_tokens=True)
    answer_tokens_to_string = tokenizer.convert_tokens_to_string(answer_tokens)
    if verbose:
        print ("\nQuestion: ",question)
        print ("Answer: ", answer_tokens_to_string)
    return [question, answer_tokens_to_string]


tokenizer = AutoTokenizer.from_pretrained('distilbert-base-cased-distilled-squad')
model = AutoModelForQuestionAnswering.from_pretrained('distilbert-base-cased-distilled-squad')

query(context=context, question=question)

/Users/venkat/opt/anaconda3/envs/llm/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



Question:  what is the urban agglomeration of Bangalore?
Answer:  10, 456, 000


['what is the urban agglomeration of Bangalore?', '10, 456, 000']

In [17]:
question = "What is Computer vision?"

def query(context, question, verbose=True, context_max_len=2500):
    if len(context) > context_max_len:
        if verbose:
            print(f'Warning: Context length {len(context)} exceeds max {context_max_len}, truncating')
            context = context[:context_max_len]
    # print (context)
    encoding = tokenizer.encode_plus(question, context)
    input_ids, attention_mask = encoding["input_ids"], encoding["attention_mask"]
    ret = model(torch.tensor([input_ids]), attention_mask=torch.tensor([attention_mask]))  # bug work-around
    start_scores, end_scores = ret[0], ret[1]
    ans_tokens = input_ids[torch.argmax(start_scores) : torch.argmax(end_scores)+1]
    answer_tokens = tokenizer.convert_ids_to_tokens(ans_tokens , skip_special_tokens=True)
    answer_tokens_to_string = tokenizer.convert_tokens_to_string(answer_tokens)
    if verbose:
        print ("\nQuestion: ",question)
        print ("Answer: ", answer_tokens_to_string)
    return [question, answer_tokens_to_string]

query(context=context, question=question)


Question:  What is Computer vision?
Answer:  an interdisciplinary field that deals with how computers can be made to gain high - level understanding from digital images or videos


['What is Computer vision?',
 'an interdisciplinary field that deals with how computers can be made to gain high - level understanding from digital images or videos']

In [18]:
query(context=context, question="what are uses of computer vision?")


Question:  what are uses of computer vision?
Answer:  fashion eCommerce, inventory management, patent search, furniture, and the beauty industry


['what are uses of computer vision?',
 'fashion eCommerce, inventory management, patent search, furniture, and the beauty industry']

In [ ]:
query(context=context, question="what is field of computer vision?")